In [1]:
import sys
import subprocess
import cdsapi
import os
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

import errno
from dask.diagnostics import ProgressBar, progress
import scipy
import netCDF4
from dask.distributed import wait, progress

In [2]:
from dask.distributed import Client

client = Client(n_workers=56,dashboard_address=':8788', threads_per_worker=2, memory_limit='4GB')
#client = Client('tcp://127.0.0.1:58098')
client

Client Scheduler: tcp://127.0.0.1:42525 Dashboard: http://127.0.0.1:8788/status,Cluster Workers: 56 Cores: 112 Memory: 224.00 GB


In [3]:
# Load the country boundaries file
countries_path = r"./data"
countries_path = os.path.join(countries_path, "ne_10m_admin_0_countries_lakes.shp")
states_path = os.path.join(r"./data", "ne_10m_admin_1_states_provinces_lakes.shp")

# Load the state boundaries file and filter for CONUS
states_gdf = gpd.read_file(states_path)
states_gdf.head()
countries_gdf = gpd.read_file(countries_path)
SARBnd = states_gdf[states_gdf['iso_a2'].isin(['PE','CO','EC','BO'])]

In [4]:
# Pass a GeoPandas dataframe and get back the bounding box
def GetBounds(zone_aoi):
    # Get lat min, max
    aoi_lat = [float(zone_aoi.total_bounds[1]), float(zone_aoi.total_bounds[3])]
    aoi_lon = [float(zone_aoi.total_bounds[0]), float(zone_aoi.total_bounds[2])]
    # Notice that the longitude values have negative numbers
    # we need these values in a global crs so we can subtract from 360
    return aoi_lat, aoi_lon

In [ ]:
DoIt = True
CalcWS = True
mychunk = {'latitude':25,'longitude':25,'time': -1}
if DoIt:
    def RunWS(zone,zone_gdf):
        aoi_lat, aoi_lon = GetBounds(zone_gdf)
        # The netcdf files use a global lat/lon so adjust values accordingly
        aoi_lon[0] = aoi_lon[0]# + 360
        aoi_lon[1] = aoi_lon[1] #+ 360
        basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
        #basedir = '/mnt/nas/jolly'
        yrs = range(2010,2015)
        mons = range(1,13)
        for y in yrs:
            for m in mons:
                print(y,m)
                #xr.open_mfdataset('my/files/*.nc', parallel=True)
                # Note: need to save air temparature grid after clipping
                uwindfullpath = '%s/era5-land-10m_u_component_of_wind-%s-%02d.nc' % (basedir,y,m)
                vwindfullpath = '%s/era5-land-10m_v_component_of_wind-%s-%02d.nc' % (basedir,y,m)

                ds_uwind = xr.open_dataset(uwindfullpath,chunks=mychunk,engine='netcdf4')
                ds_vwind = xr.open_dataset(vwindfullpath,chunks=mychunk,engine='netcdf4')
            
                print("Opened all the datasets")
                # Reassign the coordinates to -180 to 180 from 0 to 360
                
                ds_uwind = ds_uwind.assign_coords(longitude=(((ds_uwind.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_vwind = ds_vwind.assign_coords(longitude=(((ds_vwind.longitude + 180) % 360) - 180)).sortby('longitude')

                # Subset data for zone
                zone_uwind = ds_uwind.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))
                zone_vwind = ds_vwind.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))
                print("Writing out the clipped U and V")
                
                # Write the clipped U/V winds file
                ofile = '%s/era5-land-%s-0m_u_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                zone_uwind.to_netcdf(ofile,format= 'NETCDF4' )

                ofile = '%s/era5-land-%s-0m_v_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                zone_vwind.to_netcdf(ofile,format= 'NETCDF4')

                if CalcWS:
                    print("Calculating WS")
                    ifile = '%s/era5-land-%s-0m_u_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_uwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    ifile = '%s/era5-land-%s-0m_v_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_vwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    
                    outds = zone_uwind
                    outds = outds.drop('u10')
                    outds.attrs['Desc'] = "Derived from 10m U and V component winds from ERA5-Land"
                    outds.attrs['units'] = "m/s"
                    outds.attrs['long_name'] = "10 metre windspeed"
                    ofile = '%s/era5-land-%s_10m_ws-%s-%02d.nc' % (basedir,zone,y,m)
                    print("Outfile: %s:" % (ofile))
                    outds['ws'] = np.sqrt(zone_uwind.u10**2 + zone_vwind.v10**2)
                    outds.to_netcdf(ofile,format= 'NETCDF4')
                    
                #if CalcWS:
                #    print("Calculating WS")
                #    ifile = '%s/era5-land-%s-0m_u_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                #    zone_uwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                #    ifile = '%s/era5-land-%s-0m_v_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                #    zone_vwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                #    
                #    outds = zone_uwind
                #    outds = outds.drop('u10')
                #    outds.attrs['Desc'] = "Derived from 10m U and V component winds from ERA5-Land"
                #    outds.attrs['units'] = "m/s"
                #    outds.attrs['long_name'] = "10 metre windspeed"
                #    ofile = '%s/era5-land-%s-10m_ws-%s-%02d.nc' % (basedir,zone,y,m)
                #    print("Outfile: %s:" % (ofile))
                #    ws_grid = calc_ws(zone_uwind.u10,zone_vwind.v10)
                
                #    with ProgressBar():
                #        outds['ws'] = ws_grid.compute()
                    #print(zone_uwind)
                #        outds.to_netcdf(ofile,format= 'NETCDF4' )

    %time RunWS("SAR",SARBnd)

In [ ]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunWSMax(zone):
    #zone = "SAR"
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    #basedir = '/mnt/nas/jolly'
    yrs = range(2015,2021)
    for y in yrs:
        print("Summarizing max daily windspeed for year:",y)
        #era5-land-SAR_10m_ws-2017-09.nc
        ifile = '%s/era5-land-%s_10m_ws-%s-*nc' % (basedir,zone,y)
        print("Outfile: %s:" % (ifile))
        zone_temp= xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        outds = zone_temp.resample(time='D').max('time')
        ifile = '%s/era5-land-%s-10m_wsmax-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )

%time RunWSMax("SAR")

In [ ]:
def calc_rh_gufunc(tair, tdew):
    x = np.exp(1.81 + (tair * 17.27 - 4717.31) / (tair - 35.86))
    y = np.exp(1.81 + (tdew * 17.27 - 4717.31) / (tdew - 35.86))
    return ((y/x) * 100)

def calc_rh_ts(tair,tdew):
    
    return calc_rh_vect(tair,tdew)
    
    
calc_rh_vect= np.vectorize(calc_rh_gufunc)

def calc_rh(x, y):
    return xr.apply_ufunc(
        #calc_rh_gufunc,
        calc_rh_ts,
        x,
        y,
        dask="parallelized",
        input_core_dims=[["time"],["time"]], 
        #output_core_dims=[["latitude"],["longitude"]],
        vectorize=True, # !Importat!
        output_dtypes=[x.dtype],
    )

def calc_rh(x, y):
    return xr.apply_ufunc(
        calc_rh_gufunc,
        x,
        y,
        dask="parallelized",
        vectorize=True, # !Important!
        output_dtypes=[x.dtype],
    )

In [7]:
CalcRH = True
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunTemps(zone,zone_gdf):
    aoi_lat, aoi_lon = GetBounds(zone_gdf)
    # The netcdf files use a global lat/lon so adjust values accordingly
    aoi_lon[0] = aoi_lon[0]# + 360
    aoi_lon[1] = aoi_lon[1] #+ 360
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    yrs = range(2010,2015)
    mons = range(1,13)
    for y in yrs:
        for m in mons:
            print(y,m)
            #xr.open_mfdataset('my/files/*.nc', parallel=True)
            # Note: need to save air temparature grid after clipping
            t2mfullpath = '%s/era5-land-2m_temperature-%s-%02d.nc' % (basedir,y,m)
            d2mfullpath = '%s/era5-land-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,y,m)
            print(t2mfullpath)
            
            ds_t2m = xr.open_dataset(t2mfullpath,chunks=mychunk,engine='netcdf4')
            ds_tdew = xr.open_dataset(d2mfullpath,chunks=mychunk,engine='netcdf4')
            
            print("Opened all the datasets")
            # Reassign the coordinates to -180 to 180 from 0 to 360
            ds_t2m = ds_t2m.assign_coords(longitude=(((ds_t2m.longitude + 180) % 360) - 180)).sortby('longitude')
            ds_tdew = ds_tdew.assign_coords(longitude=(((ds_tdew.longitude + 180) % 360) - 180)).sortby('longitude')
            print("Reassigned all the coordinates")

            # Subset data for zone
            zone_t2m = ds_t2m.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))
            zone_d2m = ds_tdew.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))
           
            
            print("Writing out the clipped temperature")
            # Write the clipped 2m air temperature file
            #era5-land-2m_temperature-%s-%s.nc#
            ofile = '%s/era5-land-%s-2m_temperature-%s-%02d.nc' % (basedir,zone,y,m)
            zone_t2m.to_netcdf(ofile,format= 'NETCDF4')
            ofile = '%s/era5-land-%s-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,zone,y,m)
            zone_d2m.to_netcdf(ofile,format= 'NETCDF4' )
                        
            
            if CalcRH:
                print("Calculating RH")
                ifile = '%s/era5-land-%s-2m_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                zone_t2m = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
                ifile = '%s/era5-land-%s-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                zone_d2m = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
                outds = zone_t2m
                outds = outds.drop('t2m')
                outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
                outds.attrs['units'] = "%"
                outds.attrs['long_name'] = "2 metre relative humidity"
                ofile = '%s/era5-land-%s-2m_rh-%s-%02d.nc' % (basedir,zone,y,m)
                print("Outfile: %s:" % (ofile))
                myx = np.exp(1.81 + (zone_t2m.t2m * 17.27 - 4717.31) / (zone_t2m.t2m - 35.86))
                myy = np.exp(1.81 + (zone_d2m.d2m * 17.27 - 4717.31) / (zone_d2m.d2m - 35.86))
                #rh_grid = calc_rh(zone_t2m.t2m,zone_d2m.d2m)
                outds['rh'] = (myy/myx) * 100 #rh_grid.persist()
                outds.to_netcdf(ofile,format= 'NETCDF4' )
                
%time RunTemps("SAR",SARBnd)

2010 1
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2010-01.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2010-01.nc:
2010 2
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2010-02.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2010-02.nc:
2010 3
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2010-03.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2010-03.nc:
2010 4
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2010-04.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/D

2012 8
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2012-08.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2012-08.nc:
2012 9
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2012-09.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2012-09.nc:
2012 10
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2012-10.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2012-10.nc:
2012 11
/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2012-11.nc
Opened all the datasets
Reassigned all the coordinates
Writing out the clipped temperature
Calculating RH
Outfile: /mnt

In [11]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunRHExtremes(zone):
    zone = "SAR"
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    #basedir = '/mnt/nas/jolly'
    yrs = range(2010,2015)
    for y in yrs:
        print("Summarizing RH for year:",y)
        ifile = '%s/era5-land-%s-2m_rh-%s-*nc' % (basedir,zone,y)
        print("Outfile: %s:" % (ifile))
        zone_rh = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        outds = zone_rh.resample(time='D').min('time')
        ifile = '%s/era5-land-%s-2m_rhmin-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )
        outds = zone_rh.resample(time='D').max('time')
        ifile = '%s/era5-land-%s-2m_rhmax-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )

%time RunRHExtremes("SAR")

Summarizing RH for year: 2010
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2010-*nc:
Summarizing RH for year: 2011
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2011-*nc:
Summarizing RH for year: 2012
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2012-*nc:
Summarizing RH for year: 2013
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2013-*nc:
Summarizing RH for year: 2014
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_rh-2014-*nc:
CPU times: user 1h 15min 37s, sys: 16min 20s, total: 1h 31min 58s
Wall time: 1h 52min 59s


In [12]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunTempExtremes(zone):
    #zone = "SAR"
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    #basedir = '/mnt/nas/jolly'
    yrs = range(2010,2015)
    for y in yrs:
        print("Summarizing Temperatures for year:",y)
        ifile = '%s/era5-land-%s-2m_temperature-%s-*nc' % (basedir,zone,y)
        print("Outfile: %s:" % (ifile))
        zone_temp= xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        outds = zone_temp.resample(time='D').min('time')
        ifile = '%s/era5-land-%s-2m_tmin-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )
        outds = zone_temp.resample(time='D').max('time')
        ifile = '%s/era5-land-%s-2m_tmax-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )

%time RunTempExtremes("SAR")

Summarizing Temperatures for year: 2010
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2010-*nc:
Summarizing Temperatures for year: 2011
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2011-*nc:
Summarizing Temperatures for year: 2012
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2012-*nc:
Summarizing Temperatures for year: 2013
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2013-*nc:
Summarizing Temperatures for year: 2014
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2014-*nc:
CPU times: user 1h 17min 2s, sys: 18min 21s, total: 1h 35min 23s
Wall time: 1h 56min 7s


In [8]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunPrcpSrad(zone,zone_gdf):
    aoi_lat, aoi_lon = GetBounds(zone_gdf)
    # The netcdf files use a global lat/lon so adjust values accordingly
    aoi_lon[0] = aoi_lon[0]# + 360
    aoi_lon[1] = aoi_lon[1] #+ 360
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    yrs = range(2010,2015)
    mons = range(1,13)
    for y in yrs:
        for m in mons:
            print(y,m)         
            srfullpath = '%s/era5-land-surface_solar_radiation_downwards-%s-%02d.nc' % (basedir,y,m)
            prfullpath = '%s/era5-land-total_precipitation-%s-%02d.nc' % (basedir,y,m)

            #ds_sr = xr.open_dataset(srfullpath,chunks=mychunk)
            ds_pr = xr.open_dataset(prfullpath,chunks=mychunk)

            print("Opened all the datasets")
            # Reassign the coordinates to -180 to 180 from 0 to 360
            
            #ds_sr = ds_sr.assign_coords(longitude=(((ds_sr.longitude + 180) % 360) - 180)).sortby('longitude')
            ds_pr = ds_pr.assign_coords(longitude=(((ds_pr.longitude + 180) % 360) - 180)).sortby('longitude')

            #zone_sr = ds_sr.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))
            zone_pr = ds_pr.sel(longitude=slice(aoi_lon[0], aoi_lon[1]),latitude=slice(aoi_lat[1], aoi_lat[0]))

            #ofile = '%s/era5-land-%s-surface_solar_radiation_downwards-%s-%02d.nc' % (basedir,zone,y,m)
            #zone_sr.to_netcdf(ofile,format= 'NETCDF4' )
            print("Saving hourly precip for %s for domain %s" % (y,zone))
            ofile = '%s/era5-land-%s-total_precipitation-%s-%02d.nc' % (basedir,zone,y,m)
            zone_pr.to_netcdf(ofile,format= 'NETCDF4' )

%time RunPrcpSrad("SAR",SARBnd)           

2010 1
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 2
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 3
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 4
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 5
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 6
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 7
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 8
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 9
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 10
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 11
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2010 12
Opened all the datasets
Saving hourly precip for 2010 for domain SAR
2011 1
Opened all the datasets
Saving hourly precip for 2011 for domain SAR
2011 2
Op

In [13]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunPrcpDailyTotal(zone):
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    yrs = range(2010,2015)
    for y in yrs:
        print("Summarizing Precipitation for year:",y)
        ifile = '%s/era5-land-%s-total_precipitation-%s*.nc' % (basedir,zone,y)
        print("Input file: %s:" % (ifile))
        zone_prcp = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        #outds = zone_prcp.groupby('time.dayofyear').max('time')
        #zone_prcp.coords['time']  = zone_prcp.time.dt.ceil('1D')
        #ds3_pt= ds2_pt.resample(time="1D").max('time')
        #outds = zone_prcp.resample(time='D').max('time')
        mydat = zone_prcp.where(zone_prcp.time.dt.hour == 0, drop=True)
        ifile = '%s/era5-land-%s-daily_prcp-%s.nc' % (basedir,zone,y)
        mydat.to_netcdf(ifile,format= 'NETCDF4' )
        
        

%time RunPrcpDailyTotal("SAR")

Summarizing Precipitation for year: 2010
Input file: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-total_precipitation-2010*.nc:
Summarizing Precipitation for year: 2011
Input file: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-total_precipitation-2011*.nc:
Summarizing Precipitation for year: 2012
Input file: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-total_precipitation-2012*.nc:
Summarizing Precipitation for year: 2013
Input file: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-total_precipitation-2013*.nc:
Summarizing Precipitation for year: 2014
Input file: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-total_precipitation-2014*.nc:
CPU times: user 2min 33s, sys: 33.8 s, total: 3min 6s
Wall time: 3min 22s


In [ ]:
def calc_ws_gufunc(x, y,length=0):

    return np.sqrt(x**2 + y**2)


def calc_ws(x, y):
    return xr.apply_ufunc(
        calc_ws_gufunc,
        x,
        y,
        dask="parallelized",
        vectorize=True, # !Important!
        output_dtypes=[x.dtype],
    )

In [ ]:
# If the data directory doesn't exist, create it.
import os
try:
    os.makedirs('data')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
# Download the Natural Earth admin files for levels 0 and 1
import urllib.request
import zipfile

url = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_lakes.zip'
file_name = "ne_10m_admin_0_countries_lakes.zip"
response = urllib.request.urlretrieve(url, file_name)
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("data")
    
url = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces_lakes.zip'
file_name = "ne_10m_admin_1_states_provinces_lakes.zip"
response = urllib.request.urlretrieve(url, file_name)
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("data")


In [ ]:
import cfgrib
import xarray as xr
myfile = '/mnt/DataDrive2/data/mjolly/era5-land/era5-land-2m_temperature-2018-01.nc'
ds = xr.open_mfdataset(myfile,chunks={'time':-1},engine='netcdf4')

In [ ]:
ds


In [ ]:
# Loop through input files and preprocess
# This includes reading in the data and subsetting by a bounding box
# You can use any GeoPandas shape to get the bounding box

# To Do: Export the clipped air temperature data for zone
CalcRH = True
CalcWS = True

DoIt = True
if DoIt:
    
    def Run(zone,zone_gdf):
        aoi_lat, aoi_lon = GetBounds(zone_gdf)
        # The netcdf files use a global lat/lon so adjust values accordingly
        aoi_lon[0] = aoi_lon[0]# + 360
        aoi_lon[1] = aoi_lon[1] #+ 360
        #basedir = '/home/jovyan/cephfs/era5-land'
        basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
        #basedir = '/mnt/nas/jolly'
        yrs = range(2010,2015)
        mons = range(1,13)
        for y in yrs:
            for m in mons:
                print(y,m)
                #xr.open_mfdataset('my/files/*.nc', parallel=True)
                # Note: need to save air temparature grid after clipping
                t2mfullpath = '%s/era5_land_2m_temperature_%s_%02d.nc' % (basedir,y,m)
                d2mfullpath = '%s/era5_land_2m_dewpoint_temperature_%s_%02d.nc' % (basedir,y,m)
                uwindfullpath = '%s/era5_land_10m_u_component_of_wind_%s_%02d.nc' % (basedir,y,m)
                vwindfullpath = '%s/era5_land_10m_v_component_of_wind_%s_%02d.nc' % (basedir,y,m)
                srfullpath = '%s/era5_land_surface_solar_radiation_downwards_%s_%02d.nc' % (basedir,y,m)
                prfullpath = '%s/era5_land_total_precipitation_%s_%02d.nc' % (basedir,y,m)

                print(t2mfullpath)

                ds_t2m = xr.open_dataset(t2mfullpath,chunks={'latitude':20,'longitude':20,'time': -1})
                ds_tdew = xr.open_dataset(d2mfullpath,chunks={'latitude':20,'longitude':20,'time': -1})
                ds_uwind = xr.open_dataset(uwindfullpath,chunks={'latitude':20,'longitude':20,'time': -1})
                ds_vwind = xr.open_dataset(vwindfullpath,chunks={'latitude':20,'longitude':20,'time': -1})
                ds_sr = xr.open_dataset(srfullpath,chunks={'latitude':20,'longitude':20,'time': -1})
                ds_pr = xr.open_dataset(prfullpath,chunks={'latitude':20,'longitude':20,'time':-1})

                print("Opened all the datasets")
                # Reassign the coordinates to -180 to 180 from 0 to 360
                ds_t2m = ds_t2m.assign_coords(longitude=(((ds_t2m.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_tdew = ds_tdew.assign_coords(longitude=(((ds_tdew.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_uwind = ds_uwind.assign_coords(longitude=(((ds_uwind.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_vwind = ds_vwind.assign_coords(longitude=(((ds_vwind.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_sr = ds_sr.assign_coords(longitude=(((ds_sr.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_pr = ds_pr.assign_coords(longitude=(((ds_pr.longitude + 180) % 360) - 180)).sortby('longitude')

                print("Reassigned all the coordinates")
                # Slice the data by time and spatial extent
                start_date = "2019-01-15"
                end_date = "2019-01-16"

                # Subset data for zone
                zone_t2m = ds_t2m.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_d2m = ds_tdew.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_uwind = ds_uwind.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_vwind = ds_vwind.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_sr = ds_sr.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_pr = ds_pr.sel(
                    #time=slice(start_date,end_date),
                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_t2m.chunk({'latitude':20,'longitude':20,'time': -1})
                zone_d2m.chunk({'latitude':20,'longitude':20,'time': -1})
                zone_uwind.chunk({'latitude':20,'longitude':20,'time': -1})
                zone_vwind.chunk({'latitude':20,'longitude':20,'time': -1})
                zone_sr.chunk({'latitude':20,'longitude':20,'time': -1})
                zone_pr.chunk({'latitude':20,'longitude':20,'time': -1})

                print("Writing out the clipped temperature")
                # Write the clipped 2m air temperature file
                ofile = '%s/era5_land_%s_2m_temperature_%s_%s.nc' % (basedir,zone,y,m)
                delwrite = zone_t2m.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' ,compute=False)
                with ProgressBar():
                    delwrite.compute()
                print("Writing out the clipped solar radiation and precipitation")
                # Write out the clipped met files
                ofile = '%s/era5_land_%s_surface_solar_radiation_downwards_%s_%02d.nc' % (basedir,zone,y,m)
                zone_sr.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )
                ofile = '%s/era5_land_%s_total_precipitation_%s_%02d.nc' % (basedir,zone,y,m)
                zone_pr.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )


                if CalcRH:
                    print("Calculating RH")
                    outds = zone_t2m
                    outds = outds.drop('t2m')
                    outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
                    outds.attrs['units'] = "%"
                    outds.attrs['long_name'] = "2 metre relative humidity"
                    ofile = '%s/era5_land_%s_2m_rh_%s_%02d.nc' % (basedir,zone,y,m)
                    print("Outfile: %s:" % (ofile))
                    rh_grid = calc_rh(zone_t2m.t2m,zone_d2m.d2m)
                    with ProgressBar():
                        outds['rh'] = rh_grid.compute()


                        outds.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )
                if CalcWS:
                    print("Calculating WS")
                    outds = zone_t2m
                    outds = outds.drop('t2m')
                    outds.attrs['Desc'] = "Derived from 10m U and V component winds from ERA5-Land"
                    outds.attrs['units'] = "m/s"
                    outds.attrs['long_name'] = "10 metre windspeed"
                    ofile = '%s/era5_land_%s_2m_ws_%s_%s.nc' % (basedir,zone,y,m)
                    print("Outfile: %s:" % (ofile))
                    ws_grid = calc_ws(zone_uwind.u10,zone_vwind.v10)

                    with ProgressBar():
                        outds['ws'] = ws_grid.compute()
                    #print(zone_uwind)
                        outds.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )

    %time Run("SAR",SARBnd)

In [ ]:
CalcRH = True
DoIt = False

mychunk = {'latitude':25,'longitude':25,'time': -1}
if DoIt:
    def RunTemps(zone,zone_gdf):
        aoi_lat, aoi_lon = GetBounds(zone_gdf)
        # The netcdf files use a global lat/lon so adjust values accordingly
        aoi_lon[0] = aoi_lon[0]# + 360
        aoi_lon[1] = aoi_lon[1] #+ 360
        basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
        #basedir = '/mnt/nas/jolly'
        yrs = range(2015,2021)
        mons = range(1,13)
        for y in yrs:
            for m in mons:
                print(y,m)
                #xr.open_mfdataset('my/files/*.nc', parallel=True)
                # Note: need to save air temparature grid after clipping
                t2mfullpath = '%s/era5-land-2m_temperature-%s-%02d.nc' % (basedir,y,m)
                d2mfullpath = '%s/era5-land-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,y,m)
                print(t2mfullpath)

                ds_t2m = xr.open_dataset(t2mfullpath,chunks=mychunk,engine='netcdf4')
                ds_tdew = xr.open_dataset(d2mfullpath,chunks=mychunk,engine='netcdf4')

                print("Opened all the datasets")
                # Reassign the coordinates to -180 to 180 from 0 to 360
                ds_t2m = ds_t2m.assign_coords(longitude=(((ds_t2m.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_tdew = ds_tdew.assign_coords(longitude=(((ds_tdew.longitude + 180) % 360) - 180)).sortby('longitude')
                print("Reassigned all the coordinates")


                # Subset data for zone
                zone_t2m = ds_t2m.sel(

                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))

                zone_d2m = ds_tdew.sel(

                    longitude=slice(aoi_lon[0], aoi_lon[1]),
                    latitude=slice(aoi_lat[1], aoi_lat[0]))



                #zone_t2m = zone_t2m.chunk({'latitude':1,'longitude':1,'time': -1})
                #zone_d2m = zone_d2m.chunk({'latitude':1,'longitude':1,'time': -1})


                print("Writing out the clipped temperature")
                # Write the clipped 2m air temperature file
                #era5-land-2m_temperature-%s-%s.nc#
                ofile = '%s/era5-land-%s-2m_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                delwrite = zone_t2m.to_netcdf(ofile,format= 'NETCDF4' ,compute=False)
                from dask.distributed import progress

                result = delwrite.persist()
                progress(result.compute())



                ofile = '%s/era5-land-%s-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                delwrite = zone_d2m.to_netcdf(ofile,format= 'NETCDF4' ,compute=False)
                result = delwrite.persist()
                progress(result.compute())


                if CalcRH:
                    print("Calculating RH")
                    ifile = '%s/era5-land-%s-2m_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_t2m = xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    ifile = '%s/era5-land-%s-2m_dewpoint_temperature-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_d2m = xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    outds = zone_t2m
                    outds = outds.drop('t2m')
                    outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
                    outds.attrs['units'] = "%"
                    outds.attrs['long_name'] = "2 metre relative humidity"
                    ofile = '%s/era5-land-%s-2m_rh-%s-%02d.nc' % (basedir,zone,y,m)
                    print("Outfile: %s:" % (ofile))
                    rh_grid = calc_rh(zone_t2m.t2m,zone_d2m.d2m)
                    outds['rh'] = rh_grid.persist()
                    with ProgressBar():
                        outds.to_netcdf(ofile,format= 'NETCDF4' )

    %time RunTemps("SAR",SARBnd)

In [ ]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunPrcpDailyTotal(zone):
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    yrs = range(2015,2021)
    for y in yrs:
        print("Summarizing Precipitation for year:",y)
        ifile = '%s/era5-land-%s-total_precipitation-%s*.nc' % (basedir,zone,y)
        print("Input file: %s:" % (ifile))
        zone_prcp = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        #outds = zone_prcp.groupby('time.dayofyear').max('time')
        #zone_prcp.coords['time']  = zone_prcp.time.dt.ceil('1D')
        #ds3_pt= ds2_pt.resample(time="1D").max('time')
        #outds = zone_prcp.resample(time='D').max('time')
        mydat = zone_prcp.where(zone_prcp.time.dt.hour == 0, drop=True)
        ifile = '%s/era5-land-%s-daily_prcp-%s.nc' % (basedir,zone,y)
        mydat.to_netcdf(ifile,format= 'NETCDF4' )
        
        

%time RunPrcpDailyTotal("SAR")

In [ ]:
basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
y=2017
zone="SAR"
ifile = '%s/era5-land-%s-daily_prcp-*.nc' % (basedir,zone)
ds = xr.open_mfdataset(ifile,parallel=True,engine='netcdf4' )
ifile = '%s/era5-land-%s-total_precipitation-%s*.nc' % (basedir,zone,y)
ds2 = xr.open_mfdataset(ifile,engine='netcdf4')

In [ ]:
ds.coords['time']  = ds.time.dt.ceil('1D')
dsann = ds.groupby('time.year').sum('time')


In [ ]:
dsann.tp[3].plot()

In [ ]:
def special_max(x):
    print(x)
    return np.max(x)
    

def special_func(data):
    return xr.apply_ufunc(special_max, data, input_core_dims=[["time"]],
                          #output_core_dims=[["time"]],
             dask = 'allowed', vectorize = True)
ds2_pt = ds2.sel(latitude=-11.58257164274016,longitude=-70.61278378627453,method='nearest')
new = ds2_pt.tp[0:200].resample(time="1D").apply(special_func).compute()
new = ds2_pt.resample(time="24H").reduce(np.max).compute()

In [ ]:
###############################################################################
# Super important!!!!!!!
###############################################################################
ds_pt = ds.sel(latitude=-11.58257164274016,longitude=-70.61278378627453,method='nearest')
ds2_pt = ds2.sel(latitude=-11.58257164274016,longitude=-70.61278378627453,method='nearest')
ds2_pt.coords['time']  = ds2_pt.time.dt.ceil('1D')
ds3_pt= ds2_pt.resample(time="1D").max('time')

#ds3_pt = ds2_pt.resample(time="1D",label="left").max()
#ds_pt.tp[1:4].plot(label="DS")
#ds2_pt.tp[100:200].plot()
#ds3_pt.tp[4:10].plot()
#lt.legend()
ds2_pt.tp[2400:3000].plot()
ds3_pt.tp[100:120].plot()
ds3_pt.tp.sum().compute()

In [ ]:
ds2_pt.time.dt.floor('1D')

In [ ]:
ds3_pt.tp.plot()

In [ ]:
ds.tp.sum('time').plot.hist()

In [ ]:
basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
zone = "SAR"
y = 2018
m = 1
mychunk = {'latitude':20,'longitude':20,'time': -1}
ifile = '%s/era5-land-%s-0m_u_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
zone_uwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')

In [ ]:
# Assumes data are already clipped
CalcRH = True
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunDailySummaries(zone):
    
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    #basedir = '/mnt/nas/jolly'
    yrs = range(2018,2019)
    mons = range(1,13)
    for y in yrs:
        for m in mons:
            print(y,m)
            # Open the temperature and RH files
            ifile = '%s/era5-land-%s-2m_temperature-%s-%02d.nc' % (basedir,zone,y,m)
            zone_t2m = xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
            ofile = '%s/era5-land-%s-2m_rh-%s-%02d.nc' % (basedir,zone,y,m)
            zone_rh = xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
            print(zone_rh)

 
                
%time RunDailySummaries("SAR")

In [ ]:
aoi_lat, aoi_lon = GetBounds(SARBnd)
# The netcdf files use a global lat/lon so adjust values accordingly
aoi_lon[0] = aoi_lon[0]# + 360
aoi_lon[1] = aoi_lon[1] #+ 360
basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
#basedir = '/mnt/nas/jolly/era5-land'
mychunk = {'latitude':25,'longitude':25,'time': -1}

zone = "SAR"



ifile = '%s/era5-land-%s-2m_temperature-201*.nc' % (basedir,zone)
zone_t2m = xr.open_mfdataset(ifile,parallel=True )
zone_t2m = zone_t2m.chunk({'latitude':20,'longitude':20,'time': -1})
ifile = '%s/era5-land-%s-2m_rh-201*.nc' % (basedir,zone)
zone_rh = xr.open_mfdataset(ifile,parallel=True )
zone_rh = zone_rh.chunk({'latitude':20,'longitude':20,'time': -1})
ifile = '%s/era5-land-%s-total_precipitation-201*.nc' % (basedir,zone)
zone_prcp = xr.open_mfdataset(ifile,parallel=True )
zone_prcp = zone_prcp.chunk({'latitude':20,'longitude':20,'time': -1})
ifile = '%s/era5-land-%s-surface_solar_radiation_downwards-201*.nc' % (basedir,zone)
zone_srad = xr.open_mfdataset(ifile,parallel=True )
zone_srad = zone_srad.chunk({'latitude':20,'longitude':20,'time': -1})
ifile = '%s/era5-land-%s-2m_ws-201*.nc' % (basedir,zone)
zone_ws = xr.open_mfdataset(ifile,parallel=True )
zone_ws = zone_ws.chunk({'latitude':20,'longitude':20,'time': -1})

zone_t2m_pt = zone_t2m.sel(longitude=-70.82254283405304,latitude=-11.75684334608126,method='nearest').persist()
zone_srad_pt = zone_srad.sel(longitude=-70.82254283405304,latitude=-11.75684334608126,method='nearest').persist()
zone_rh_pt = zone_rh.sel(longitude=-70.82254283405304,latitude=-11.75684334608126,method='nearest').persist()
zone_prcp_pt = zone_prcp.sel(longitude=-70.82254283405304,latitude=-11.75684334608126,method='nearest').persist()
zone_ws_pt = zone_ws.sel(longitude=-70.82254283405304,latitude=-11.75684334608126,method='nearest').persist()

In [ ]:
# A 2D plot of the SPEED variable, assigning the coordinate values,
# and plot the verticies of each point
ds.SPEED.plot(x='longitude', y='latitude')
plt.scatter(ds.longitude, ds.latitude)

# I want to find the speed at a certain lat/lon point.
lat = 21.22
lon = -122.68

# First, find the index of the grid point nearest a specific lat/lon.   
abslat = np.abs(ds.latitude-lat)
abslon = np.abs(ds.longitude-lon)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c))

# Now I can use that index location to get the values at the x/y diminsion
point_ds = ds.sel(x=xloc, y=yloc)

# Plot requested lat/lon point blue
plt.scatter(lon, lat, color='b')
plt.text(lon, lat, 'requested')

# Plot nearest point in the array red
plt.scatter(point_ds.longitude, point_ds.latitude, color='r')
plt.text(point_ds.longitude, point_ds.latitude, 'nearest')

plt.title('speed at nearest point: %s' % point_ds.SPEED.data)

In [ ]:
# Fix the hourly precip and solar radiation

import xarray as xr

def special_mean(x, drop_min=False):
    return np.diff(x,prepend=0)
    

def special_func(data):
    return xr.apply_ufunc(special_mean, data, input_core_dims=[["time"]],
                          output_core_dims=[["time"]],
            kwargs={'drop_min': True}, dask = 'allowed', vectorize = True)

# Fix the time so that the 24 hour accumulations work correctly
zone_t2m.time.where(zone_t2m.time.dt.hour == 0)
temp = zone_t2m.time
tempsave = temp
temp = xr.where((temp.dt.hour == 0), temp - np.timedelta64(1,'s'), temp)
#zone_prcp['time'] = temp
print(temp)

data = zone_srad_pt
data['time'] = temp
new = data.ssrd.resample(time='D').apply(special_func) / 3600
new[0] = 0
#new[1:48].plot()
zone_full_pt['ssrd'] = new

data = zone_prcp_pt
data['time'] = temp
new = data.tp.resample(time='D').apply(special_func)  * 100
new[0] = 0
new.plot()
zone_full_pt['tp'] = new


In [ ]:
# Loop through input files and preprocess
# This includes reading in the data and subsetting by a bounding box
# You can use any GeoPandas shape to get the bounding box


DoIt = True
mychunk = {'latitude':25,'longitude':25,'time': -1}
if DoIt:
    
    def Run(zone,zone_gdf):
        aoi_lat, aoi_lon = GetBounds(zone_gdf)
        # The netcdf files use a global lat/lon so adjust values accordingly
        aoi_lon[0] = aoi_lon[0]# + 360
        aoi_lon[1] = aoi_lon[1] #+ 360
        #basedir = '/home/jovyan/cephfs/era5-land'
        basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
        #basedir = '/mnt/nas/jolly'
        yrs = range(2015,2021)
        mons = range(1,13)
        for y in yrs:
            for m in mons:
                print(y,m)
                #xr.open_mfdataset('my/files/*.nc', parallel=True)
                # Note: need to save air temparature grid after clipping
                
                ##srfullpath = '%s/era5-land-surface_solar_radiation_downwards-%s-%02d.nc' % (basedir,y,m)
                prfullpath = '%s/era5-land-total_precipitation-%s-%02d.nc' % (basedir,y,m)

                print(prfullpath)

                
                #ds_sr = xr.open_dataset(srfullpath,chunks=mychunk,engine='netcdf4')
                ds_pr = xr.open_dataset(prfullpath,chunks=mychunk,engine='netcdf4')

                print("Opened all the datasets")
                # Reassign the coordinates to -180 to 180 from 0 to 360
                
                #ds_sr = ds_sr.assign_coords(longitude=(((ds_sr.longitude + 180) % 360) - 180)).sortby('longitude')
                ds_pr = ds_pr.assign_coords(longitude=(((ds_pr.longitude + 180) % 360) - 180)).sortby('longitude')

                print("Reassigned all the coordinates")
                # Slice the data by time and spatial extent
                

                # Subset data for zone
                
                ##zone_sr = ds_sr.sel(longitude=slice(aoi_lon[0], aoi_lon[1]), latitude=slice(aoi_lat[1], aoi_lat[0]))
                zone_pr = ds_pr.sel(longitude=slice(aoi_lon[0], aoi_lon[1]), latitude=slice(aoi_lat[1], aoi_lat[0]))
                
                #zone_sr.chunk(mychunk)
                zone_pr.chunk(mychunk)
 
                # Write out the clipped met files
                #ofile = '%s/era5_land_%s_surface_solar_radiation_downwards_%s_%02d.nc' % (basedir,zone,y,m)
                #zone_sr.to_netcdf(ofile,format= 'NETCDF4' )
                ofile = '%s/era5_land_%s_total_precipitation_%s_%02d.nc' % (basedir,zone,y,m)
                zone_pr.to_netcdf(ofile,format= 'NETCDF4' )


               
    %time Run("SAR",SARBnd)

In [ ]:
DoIt = True
CalcWS = True
mychunk = {'latitude':25,'longitude':25,'time': -1}
if DoIt:
    def RunWS(zone,zone_gdf):
        aoi_lat, aoi_lon = GetBounds(zone_gdf)
        # The netcdf files use a global lat/lon so adjust values accordingly
        aoi_lon[0] = aoi_lon[0]# + 360
        aoi_lon[1] = aoi_lon[1] #+ 360
        basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
        #basedir = '/mnt/nas/jolly'
        yrs = range(2015,2021)
        mons = range(1,13)
        for y in yrs:
            for m in mons:
                print(y,m)
   


                if CalcWS:
                    print("Calculating WS")
                    ifile = '%s/era5-land-%s-0m_u_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_uwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    ifile = '%s/era5-land-%s-0m_v_component_of_wind-%s-%02d.nc' % (basedir,zone,y,m)
                    zone_vwind= xr.open_dataset(ifile,chunks=mychunk,engine='netcdf4')
                    
                    outds = zone_uwind
                    outds = outds.drop('u10')
                    outds.attrs['Desc'] = "Derived from 10m U and V component winds from ERA5-Land"
                    outds.attrs['units'] = "m/s"
                    outds.attrs['long_name'] = "10 metre windspeed"
                    ofile = '%s/era5-land-%s_10m_ws-%s-%02d.nc' % (basedir,zone,y,m)
                    print("Outfile: %s:" % (ofile))
                    outds['ws'] = np.sqrt(zone_uwind.u10**2 + zone_vwind.v10**2)
                    outds.to_netcdf(ofile,format= 'NETCDF4')
                    

    %time RunWS("SAR",SARBnd)